In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import utide
import xarray as xr

/nbhome/ogrp/python/envs/py311_20241018/lib/python3.11/site-packages/utide/harmonics.py:16: RuntimeWarning: invalid value encountered in cast
  nshallow = np.ma.masked_invalid(const.nshallow).astype(int)
/nbhome/ogrp/python/envs/py311_20241018/lib/python3.11/site-packages/utide/harmonics.py:17: RuntimeWarning: invalid value encountered in cast
  ishallow = np.ma.masked_invalid(const.ishallow).astype(int) - 1


In [2]:
year_start=1958
year_end=2014

model_loc = ['SEATTLE', 'SANDY_HOOK', 'WELLINGTON_HARBOUR']

gesla_loc = {'SEATTLE': "seattle-9447130-usa-noaa",
		  'SANDY_HOOK': "sandy_hook-8531680-usa-noaa",
          'WELLINGTON_HARBOUR': "wellington-071a-nzl-uhslc"}

ds = xr.open_dataset("/vftmp/Olivia.Mcredmond/data/model_timeseries_odiv181.nc",decode_times=True)
da = xr.open_dataset("/vftmp/Olivia.Mcredmond/data/model_timeseries_odiv2.nc",decode_times=True)
odiv181_pressure= xr.open_dataset("/vftmp/Olivia.Mcredmond/data/model_timeseries_psl_odiv181.nc")
odiv2_pressure= xr.open_dataset("/vftmp/Olivia.Mcredmond/data/model_timeseries_psl_odiv2.nc")

In [3]:
def process_model_data(location_name, year_start,year_end):
    model_ts = ds[location_name]
    model_ta = da[location_name]
    
    model2= model_ta.sel(time=slice(f"{year_start}-01-01", f"{year_end}-12-31"))
    model2= model2-model2.mean()
    model2['time']=[pd.to_datetime(str(np.array(model2.time[i])), format='mixed', yearfirst=True) for i in range(0,len(np.array(model2)))]
    
    model181= model_ts.sel(time=slice(f"{year_start}-01-01", f"{year_end}-12-31"))
    model181=model181-model181.mean()
    model181['time']=[pd.to_datetime(str(np.array(model181.time[i])), format='mixed', yearfirst=True) for i in range(0,len(np.array(model181)))]
    
    model2_IB = -1*(odiv2_pressure[location_name].sel(time=slice(f"{year_start}-01-01", f"{year_end}-12-31")))*0.01*0.1/(1.02*9.8)
    model2_IB['time']=[pd.to_datetime(str(np.array(model2_IB.time[i])), format='mixed', yearfirst=True) for i in range(0,len(np.array(model2_IB)))]
    model2_IB = model2+(model2_IB-model2_IB.mean())
    
    model181_IB=-1*(odiv181_pressure[location_name].sel(time=slice(f"{year_start}-01-01", f"{year_end}-12-31")))*0.01*0.1/(1.02*9.8)
    model181_IB['time']=[pd.to_datetime(str(np.array(model181_IB.time[i])), format='mixed', yearfirst=True) for i in range(0,len(np.array(model181_IB)))]
    model181_IB = model181+np.array(model181_IB-model181_IB.mean())
    
    model_output=xr.Dataset(
        data_vars={"model_181":model181,
                   "model_2":model2,
                   "model_2_IB":model2_IB,
                   "model_181_IB":model181_IB},
        coords={"time":model2.time},
        attrs={'location_name':str({location_name})})
    return model_output
		## this will save ST_PETERSBURG_MODEL_PROCESSED.nc as a
## file

In [4]:
def process_gesla_data(location_name, year_start, year_end):
    gesla_name = gesla_loc[location_name]
    data = f'../data/{gesla_name}'
    with open(data) as f:
        lines = f.readlines()
    
    for row in lines:
        word = 'LATITUDE'
        if row.find(word) != -1:
            lat= float((row.split("DE ")[1]).strip())
            break
    for row in lines:
        word = 'LONGITUDE'
        if row.find(word) != -1:
            long= float((row.split("DE ")[1]).strip())
            break
    names = ["date", "hour", "sealevel", "flag", "use_flag"]
    #Read data from file
    obs = pd.read_csv(
        data,
        names=names,
        skiprows=41, # to skip the large header and documentation, would need to check whether this is the same for every time series
        skipinitialspace=True,
        delim_whitespace=True,
        na_values="-99.9999",
    )
    
    #Turn bad data to nan using flags
    good_data = obs['use_flag'] == 1
    bad_data = obs['use_flag'] == 0
    obs.loc[bad_data, "sealevel"] = np.nan
    
    #Subtract mean and define datetime
    obs["anomaly"] = obs['sealevel'] - obs['sealevel'].mean()
    obs['datetime'] = pd.to_datetime(obs['date'] + ' ' +obs['hour'])
    
    t1 = (obs['datetime'] >= f'{year_start}-1-1') & (obs['datetime'] <= f'{year_end}-12-31')
    obs_t1=obs.loc[t1]
    print("pt 1")
    coef = utide.solve(
        obs_t1.datetime,
        obs_t1.anomaly,
        lat= lat,
        method="ols",
        conf_int="MC",
        verbose=False,
       constit=['M2', 'K1', 'O1', 'S2', 'P1', 'N2', 'K2'] 
    )
    tide = utide.reconstruct(obs_t1.datetime, coef, verbose=False)
    obs_t1['sealevel_tr'] = obs_t1.anomaly - tide.h
    obs_t1['datetime'] = pd.to_datetime(obs_t1['datetime'])
    # Set the datetime column as the index
    obs_t1.set_index('datetime', inplace=True)
    # Resample from hourly to daily and calculate the daily average
    daily_avg=obs_t1['sealevel_tr'].resample('D').mean()
    daily_avg=daily_avg.to_frame()
    daily_set = xr.Dataset.from_dataframe(daily_avg)
    daily_set.attrs['location_name']=location_name
    daily_set.attrs['latitude']=lat
    daily_set.attrs['longitude']=long
    return daily_set
## You can then compare ST_PETERSBURG_MODEL_PROCESSED.nc and ST_PETERSBURG_GESLA_PROCESSED.nc!

In [5]:
for i in model_loc:
    model_output=process_model_data(i,year_start,year_end)
    obs=process_gesla_data(i,year_start,year_end)
    Location_dataset=xr.Dataset(
        data_vars={"model_181":model_output.model_181,
                   "model_2":model_output.model_2,
                   "model_2_IB":model_output.model_2_IB,
                   "model_181_IB":model_output.model_181_IB,
                   "obs":obs.sealevel_tr},
        coords={"time":model_output.time},
        attrs={'location_name':model_output.location_name,
               'latitude': obs.latitude,
               'longitude':obs.longitude})
    Location_dataset.to_netcdf(f'/vftmp/Olivia.Mcredmond/data/cdf files/{i}.nc')

/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  obs = pd.read_csv(


pt 1


/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_t1['sealevel_tr'] = obs_t1.anomaly - tide.h
/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_t1['datetime'] = pd.to_datetime(obs_t1['datetime'])
/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future ve

pt 1


/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_t1['sealevel_tr'] = obs_t1.anomaly - tide.h
/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_t1['datetime'] = pd.to_datetime(obs_t1['datetime'])
/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future ve

pt 1


/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_t1['sealevel_tr'] = obs_t1.anomaly - tide.h
/vftmp/Olivia.Mcredmond/pid155867/ipykernel_156739/1960289138.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_t1['datetime'] = pd.to_datetime(obs_t1['datetime'])
